[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colab-samples/blob/main/notebooks/basic_features/cheatsheet.ipynb)

In [7]:
%pip install rocksdict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 58.0 MB/s eta 0:00:00


In [5]:
import json, time, os, pathlib
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

API_BASE = os.getenv("DS_API_BASE", "http://localhost:8080")
ROCKS_PATH = os.getenv("ROCKS_PATH", "tmpdb")
SESSION_ID = os.getenv("DEMO_SESSION_ID", "session-1")

try:
    import rocksdb  # type: ignore
except ModuleNotFoundError:
    rocksdb = None

try:
    from rocksdict import Rdict
except ModuleNotFoundError:
    Rdict = None

class _RocksDictIterator:
    """Compat iterator that mimics python-rocksdb's iterators."""
    def __init__(self, db, mode):
        self._db = db
        self._mode = mode
        self._data = None
        self._index = 0

    def seek_to_first(self):
        if self._mode == "keys":
            self._data = sorted(self._db.keys())
        else:
            self._data = sorted(self._db.items())
        self._index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self._data is None:
            self.seek_to_first()
        if self._index >= len(self._data):
            raise StopIteration
        value = self._data[self._index]
        self._index += 1
        return value

class _RocksDictCompat:
    """Fallback wrapper when python-rocksdb is unavailable."""
    def __init__(self, path: str):
        if Rdict is None:
            raise RuntimeError(
                "No RocksDB-compatible backend available. Install 'python-rocksdb' or 'rocksdict'."
            )
        self._db = Rdict(path)

    def put(self, key, value):
        self._db[key] = value

    def iterkeys(self):
        return _RocksDictIterator(self._db, "keys")

    def iteritems(self):
        return _RocksDictIterator(self._db, "items")

def open_rocks_db(path: str):
    if rocksdb is not None:
        return rocksdb.DB(path, rocksdb.Options(create_if_missing=True))
    return _RocksDictCompat(path)




In [6]:
db = open_rocks_db(ROCKS_PATH)



RuntimeError: No RocksDB-compatible backend available. Install 'python-rocksdb' or 'rocksdict'.

In [8]:
import requests

def ds_chat(text: str):
    payload = {"text": text}
    r = requests.post(f"{API_BASE}/v1/chat", json=payload, timeout=30)
    r.raise_for_status()
    return r.json()


In [9]:
def write_to_ledger(prompt, answer):
    key = f"p-{int(time.time()*1000)}".encode()
    confidence = {
        "dxp_service_definition": "partial",
        "dxp_user_centric": "low",
        "evidence_level": "operational/anecdotal",
        "p_value_suggested": "P50"
    }
    record = {
        "prompt": prompt,
        "answer": answer,
        "ts": time.time(),
        "confidence": confidence
    }
    db.put(key, json.dumps(record).encode())
    return key


In [10]:
def draw_tree(session_id=SESSION_ID):
    clear_output(wait=True)
    g = nx.DiGraph()
    g.add_node(session_id)
    it = db.iterkeys()
    it.seek_to_first()
    for k in it:
        k_dec = k.decode()
        g.add_node(k_dec)
        g.add_edge(session_id, k_dec)
    plt.figure(figsize=(7, 5))
    nx.draw(g, with_labels=True, node_color="lightblue", node_size=1300, font_size=7)
    plt.show()


In [11]:
def send_and_show(prompt):
    resp = ds_chat(prompt)
    answer = resp.get("text") or resp.get("answer") or str(resp)
    key = write_to_ledger(prompt, answer)
    print("LLM:", answer)
    draw_tree()
    return key


In [12]:
def search_ledger(query: str):
    it = db.iteritems()
    it.seek_to_first()
    results = []
    for k, v in it:
        data = json.loads(v.decode())
        blob = (data.get("prompt", "") + " " + data.get("answer", "")).lower()
        if query.lower() in blob:
            results.append((k.decode(), data))
    return results


In [13]:
def get_all_confidence():
    it = db.iteritems()
    it.seek_to_first()
    rows = []
    for k, v in it:
        data = json.loads(v.decode())
        if "confidence" in data:
            rows.append((k.decode(), data["confidence"]))
    return rows
